In [1]:
print("Hello Tabo, Jupyter works!")


Hello Tabo, Jupyter works!


In [2]:
import pandas as pd

df = pd.read_csv("train.csv")   # ← use exact file name
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S


In [2]:
df.isnull().sum()


PassengerId      0
Survived         1
Pclass           1
Name             1
Sex              1
Age            178
SibSp            1
Parch            1
Ticket           1
Fare             1
Cabin          688
Embarked         3
dtype: int64

In [3]:
# Fill missing Age with median
df['Age'] = df['Age'].fillna(df['Age'].median())

# Fill missing Embarked with mode (most common value)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Fill missing Cabin with "Unknown"
df['Cabin'] = df['Cabin'].fillna("Unknown")

df.isnull().sum()  # check again


PassengerId    0
Survived       1
Pclass         1
Name           1
Sex            1
Age            0
SibSp          1
Parch          1
Ticket         1
Fare           1
Cabin          0
Embarked       0
dtype: int64

In [4]:
# Convert Sex to numbers
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# One-hot encode Embarked (drop the first to avoid duplicate columns)
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

# Extract Title from Name and simplify
df['Title'] = df['Name'].str.extract(r',\s*([^\.]+)\.')

df['Title'] = df['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'], 'Rare')
df['Title'] = df['Title'].replace({'Mlle':'Miss', 'Ms':'Miss', 'Mme':'Mrs'})

# One-hot encode Title
df = pd.get_dummies(df, columns=['Title'], drop_first=True)

df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Title_the Countess
0,1,0.0,3.0,"Braund, Mr. Owen Harris",0.0,22.0,1.0,0.0,A/5 21171,7.2500,Unknown,False,True,False,True,False,False,False
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1.0,38.0,1.0,0.0,PC 17599,71.2833,C85,False,False,False,False,True,False,False
2,3,1.0,3.0,"Heikkinen, Miss. Laina",1.0,26.0,0.0,0.0,STON/O2. 3101282,7.9250,Unknown,False,True,True,False,False,False,False
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1.0,35.0,1.0,0.0,113803,53.1000,C123,False,True,False,False,True,False,False
4,5,0.0,3.0,"Allen, Mr. William Henry",0.0,35.0,0.0,0.0,373450,8.0500,Unknown,False,True,False,True,False,False,False


In [5]:
# Remove outliers from Fare using the IQR method

Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

print("Lower bound:", lower)
print("Upper bound:", upper)
print("Before:", df.shape)

df = df[(df['Fare'] >= lower) & (df['Fare'] <= upper)]

print("After:", df.shape)


Lower bound: -26.724
Upper bound: 65.6344
Before: (892, 18)
After: (775, 18)


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])

df[['Age', 'Fare']].head()


,Age,Fare
0,-0.530377,-0.502445
1,0.571831,0.786845
2,-0.254825,-0.488854
3,0.365167,0.420730
4,0.365167,-0.486337


In [4]:
df = df.drop(columns=["Name", "Ticket", "Cabin"], errors="ignore")
df.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3.0,male,-0.530377,1.0,0.0,-0.502445,S
1,2,1.0,1.0,female,0.571831,1.0,0.0,0.786845,C
2,3,1.0,3.0,female,-0.254825,0.0,0.0,-0.488854,S
3,4,1.0,1.0,female,0.365167,1.0,0.0,0.420730,S
4,5,0.0,3.0,male,0.365167,0.0,0.0,-0.486337,S


In [5]:
df.to_csv("cleaned_titanic.csv", index=False)
print("Saved cleaned_titanic.csv")


Saved cleaned_titanic.csv


In [6]:
pd.set_option('display.max_rows', None)
df


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3.0,male,-0.530377,1.0,0.0,-0.502445,S
1,2,1.0,1.0,female,0.571831,1.0,0.0,0.786845,C
2,3,1.0,3.0,female,-0.254825,0.0,0.0,-0.488854,S
3,4,1.0,1.0,female,0.365167,1.0,0.0,0.420730,S
4,5,0.0,3.0,male,0.365167,0.0,0.0,-0.486337,S
5,6,0.0,3.0,male,NaN,0.0,0.0,-0.478116,Q
6,7,0.0,1.0,male,1.674039,0.0,0.0,0.395814,S
7,8,0.0,3.0,male,-1.908136,3.0,1.0,-0.224083,S
8,9,1.0,3.0,female,-0.185937,0.0,2.0,-0.424256,S
9,10,1.0,2.0,female,-1.081480,1.0,0.0,-0.042956,C


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  892 non-null    object 
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    float64
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    float64
 6   Parch        891 non-null    float64
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(6), object(3)
memory usage: 62.8+ KB
